<a href="https://colab.research.google.com/github/jallenrobern/CCMACLRL_EXAM/blob/main/Midterm_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import re

## Dataset File

In [2]:
dataset_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/train.csv?raw=true'
df = pd.read_csv(dataset_url)

In [3]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


In [5]:
df.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,5083
engine,0
transmission,0
ext_col,0
int_col,0


In [6]:
df.dropna(inplace=True)
df.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,0
engine,0
transmission,0
ext_col,0
int_col,0


In [7]:
df.describe()

,id,model_year,milage,price
count,162610.000000,162610.000000,162610.000000,1.626100e+05
mean,94197.039137,2015.158822,71105.487086,4.081699e+04
std,54467.770815,5.699531,50157.874111,7.653857e+04
min,0.000000,1974.000000,100.000000,2.000000e+03
25%,47026.250000,2012.000000,29000.000000,1.570000e+04
50%,94139.000000,2016.000000,64300.000000,2.850000e+04
75%,141421.750000,2019.000000,101000.000000,4.699900e+04
max,188532.000000,2024.000000,405000.000000,2.954083e+06


In [8]:
df.sample(20)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
67423,67423,Lexus,IS 250 Base,2010,148949,Gasoline,204.0HP 2.5L V6 Cylinder Engine Gasoline Fuel,6-Speed A/T,Silver,Black,At least 1 accident or damage reported,Yes,7390
32738,32738,Chevrolet,Camaro 1LT,2018,70700,Gasoline,335.0HP 3.6L V6 Cylinder Engine Gasoline Fuel,6-Speed M/T,White,Black,None reported,Yes,27000
62309,62309,Toyota,Camry XLE,2016,60000,Gasoline,268.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,24700
29166,29166,Chrysler,300 S,2005,111000,Gasoline,240.0HP 3.2L Straight 6 Cylinder Engine Gasoli...,A/T,Green,Black,At least 1 accident or damage reported,Yes,4800
34660,34660,Hyundai,Sonata SE,2013,157272,Gasoline,200.0HP 2.4L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Black,At least 1 accident or damage reported,Yes,15000
119310,119310,Toyota,Sequoia Limited,2013,114890,Gasoline,381.0HP 5.7L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Black,Gray,None reported,Yes,18500
101543,101543,Jeep,Liberty Sport,2004,187883,Gasoline,210.0HP 3.7L V6 Cylinder Engine Gasoline Fuel,5-Speed A/T,Silver,Beige,At least 1 accident or damage reported,Yes,21500
110262,110262,Mercedes-Benz,E-Class E 350,2014,106000,E85 Flex Fuel,302.0HP 3.5L V6 Cylinder Engine Flex Fuel Capa...,A/T,Silver,Black,At least 1 accident or damage reported,Yes,17800
34610,34610,Ford,Excursion XLT 4WD,2005,170000,Gasoline,300.0HP 6.0L 8 Cylinder Engine Gasoline Fuel,A/T,Gold,Beige,At least 1 accident or damage reported,Yes,6500
35969,35969,Lexus,LX 570 Three-Row,2017,67000,Gasoline,383.0HP 5.7L 8 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,47500


In [9]:
current_year = datetime.now().year
df['car_age'] = current_year - df['model_year'].astype(int)

In [10]:
luxury_brands = ['BMW', 'Mercedes-Benz', 'Audi', 'Tesla', 'Porsche', 'Land', 'Aston', 'Lexus', 'Lincoln', 'Lamborghini', 'Bugatti', 'Cadillac', 'Maserati', 'Genesis', ' Acura']
df['Luxurious'] = df['brand'].apply(lambda x: 'Yes' if x in luxury_brands else 'No')

In [11]:
label_encoder = LabelEncoder()
df['brand'] = label_encoder.fit_transform(df['brand'])
df['fuel_type'] = label_encoder.fit_transform(df['fuel_type'])
df['transmission'] = label_encoder.fit_transform(df['transmission'])
df['ext_col'] = label_encoder.fit_transform(df['ext_col'])
df['int_col'] = label_encoder.fit_transform(df['int_col'])
df['accident'] = label_encoder.fit_transform(df['accident'])
df['clean_title'] = label_encoder.fit_transform(df['clean_title'])
df['Luxurious'] = label_encoder.fit_transform(df['Luxurious'])

In [12]:
def extract_engine_features(engine_desc):
    hp_match = re.search(r'(\d+(\.\d+)?)HP', engine_desc)
    hp = float(hp_match.group(1)) if hp_match else None

    l_match = re.search(r'(\d+(\.\d+)?)L', engine_desc)
    displacement = float(l_match.group(1)) if l_match else None

    cyl_match = re.search(r'(\d+) Cylinder', engine_desc)
    cylinders = int(cyl_match.group(1)) if cyl_match else None

    turbocharged = 1 if 'Turbo' in engine_desc else 0

    return hp, displacement, cylinders, turbocharged

df[['HP', 'Liters', 'Cylinders', 'Turbocharged']] = df['engine'].apply(
    lambda x: pd.Series(extract_engine_features(x))
)
drop_columns = ['engine', 'model']
df.drop(drop_columns, axis=1, inplace=True)

df.fillna(df.mean(), inplace=True)

df.sample(15)

,id,brand,model_year,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,price,car_age,Luxurious,HP,Liters,Cylinders,Turbocharged
25405,25405,9,2021,23152,2,49,221,31,0,0,47500,3,0,490.000000,6.2,8.000000,0.0
84297,84297,54,2017,79000,2,16,298,10,1,0,12700,7,0,184.000000,2.0,4.000000,0.0
88273,88273,14,2020,53000,2,38,28,71,1,0,279000,4,0,375.000000,3.5,6.000000,0.0
32643,32643,26,2023,2500,2,31,28,14,1,0,96500,1,1,523.000000,4.4,8.000000,0.0
62322,62322,14,2019,44487,2,38,126,14,1,0,49999,5,0,375.000000,3.5,6.000000,0.0
179312,179312,9,2016,67500,2,16,298,14,1,0,22000,8,0,338.336712,2.5,4.000000,0.0
89440,89440,34,2013,61300,2,16,28,14,0,0,21750,11,0,227.000000,2.5,4.000000,0.0
142907,142907,9,2013,69764,1,16,298,71,0,0,21000,11,0,320.000000,5.3,8.000000,0.0
147124,147124,3,2018,38718,2,31,28,14,1,0,32500,6,1,333.000000,3.0,6.000000,0.0
26482,26482,14,2019,59678,2,3,298,14,1,0,30000,5,0,338.336712,2.3,6.373096,1.0


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 162610 entries, 0 to 188532
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            162610 non-null  int64  
 1   brand         162610 non-null  int64  
 2   model_year    162610 non-null  int64  
 3   milage        162610 non-null  int64  
 4   fuel_type     162610 non-null  int64  
 5   transmission  162610 non-null  int64  
 6   ext_col       162610 non-null  int64  
 7   int_col       162610 non-null  int64  
 8   accident      162610 non-null  int64  
 9   clean_title   162610 non-null  int64  
 10  price         162610 non-null  int64  
 11  car_age       162610 non-null  int64  
 12  Luxurious     162610 non-null  int64  
 13  HP            162610 non-null  float64
 14  Liters        162610 non-null  float64
 15  Cylinders     162610 non-null  float64
 16  Turbocharged  162610 non-null  float64
dtypes: float64(4), int64(13)
memory usage: 22.3 MB


In [14]:
X = df.drop(['price'],axis=1)
y = df['price']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4,random_state=46)

In [15]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mean_squared_error(y_test, y_pred, squared=False)

68355.7732177658

## Test File

In [16]:
test_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/test.csv?raw=true'
dt=pd.read_csv(test_url)

In [17]:
dt.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,NaN
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes


In [18]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            125690 non-null  int64 
 1   brand         125690 non-null  object
 2   model         125690 non-null  object
 3   model_year    125690 non-null  int64 
 4   milage        125690 non-null  int64 
 5   fuel_type     122307 non-null  object
 6   engine        125690 non-null  object
 7   transmission  125690 non-null  object
 8   ext_col       125690 non-null  object
 9   int_col       125690 non-null  object
 10  accident      124058 non-null  object
 11  clean_title   111451 non-null  object
dtypes: int64(3), object(9)
memory usage: 11.5+ MB


In [19]:
dt.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,3383
engine,0
transmission,0
ext_col,0
int_col,0


In [20]:
dt.describe()

,id,model_year,milage
count,125690.000000,125690.000000,125690.000000
mean,251377.500000,2015.797526,66042.581510
std,36283.722005,5.673797,50223.858435
min,188533.000000,1974.000000,100.000000
25%,219955.250000,2013.000000,24500.000000
50%,251377.500000,2017.000000,57500.000000
75%,282799.750000,2020.000000,95798.000000
max,314222.000000,2024.000000,405000.000000


In [21]:
dt.sample(20)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
51905,240438,Pontiac,G8 GT,2005,138415,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed M/T,Black,Black,At least 1 accident or damage reported,Yes
60915,249448,Toyota,Tundra SR5,2016,165520,E85 Flex Fuel,381.0HP 5.7L 8 Cylinder Engine Flex Fuel Capab...,6-Speed A/T,Blue,–,At least 1 accident or damage reported,Yes
92602,281135,BMW,750 Li xDrive,2016,25000,Gasoline,445.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,M/T,White,Black,None reported,Yes
1534,190067,Jeep,Gladiator Overland,2021,51808,Gasoline,3.6L V6 24V MPFI DOHC,8-Speed Automatic,Firecracker Red Clearcoat,Black,None reported,Yes
58586,247119,Ford,Mustang V6 Premium,2018,80000,Gasoline,305.0HP 3.7L V6 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Black,None reported,Yes
72892,261425,Land,Rover Range Rover Velar P380 S,2013,10250,Gasoline,390.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Black,None reported,Yes
67745,256278,Ford,F-150 XLT,2020,44460,Gasoline,325.0HP 2.7L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,White,Black,None reported,Yes
120939,309472,Porsche,911 GT3,2023,8600,Gasoline,475.0HP 3.8L Flat 6 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Gray,Black,None reported,Yes
124807,313340,Jaguar,F-TYPE,2020,35248,Gasoline,2.0 Liter,Automatic,Mythos Black Metallic,Sarder Brown,None reported,NaN
76910,265443,INFINITI,G37 x,2021,22749,Gasoline,Dual Motor - Standard,Automatic,Alpine White,Black/Gun Metal,None reported,NaN


In [22]:
current_year = datetime.now().year
dt['car_age'] = current_year - df['model_year'].astype(int)

In [23]:
luxury_brands = ['BMW', 'Mercedes-Benz', 'Audi', 'Tesla', 'Porsche', 'Land', 'Aston', 'Lexus', 'Lincoln', 'Lamborghini', 'Bugatti', 'Cadillac', 'Maserati', 'Genesis', ' Acura']
dt['Luxurious'] = df['brand'].apply(lambda x: 'Yes' if x in luxury_brands else 'No')

In [24]:
label_encoder = LabelEncoder()
dt['brand'] = label_encoder.fit_transform(dt['brand'])
dt['fuel_type'] = label_encoder.fit_transform(dt['fuel_type'])
dt['transmission'] = label_encoder.fit_transform(dt['transmission'])
dt['ext_col'] = label_encoder.fit_transform(dt['ext_col'])
dt['int_col'] = label_encoder.fit_transform(dt['int_col'])
dt['accident'] = label_encoder.fit_transform(dt['accident'])
dt['clean_title'] = label_encoder.fit_transform(dt['clean_title'])
dt['Luxurious'] = label_encoder.fit_transform(dt['Luxurious'])

In [25]:
# Function to extract engine features
def extract_engine_features(engine_desc):
    # Extract horsepower (HP)
    hp_match = re.search(r'(\d+(\.\d+)?)HP', engine_desc)
    hp = float(hp_match.group(1)) if hp_match else None

    # Extract engine displacement (L)
    l_match = re.search(r'(\d+(\.\d+)?)L', engine_desc)
    displacement = float(l_match.group(1)) if l_match else None

    # Extract cylinder count
    cyl_match = re.search(r'(\d+) Cylinder', engine_desc)
    cylinders = int(cyl_match.group(1)) if cyl_match else None

    # Check if turbocharged
    turbocharged = 1 if 'Turbo' in engine_desc else 0

    return hp, displacement, cylinders, turbocharged

dt[['HP', 'Liters', 'Cylinders', 'Turbocharged']] = dt['engine'].apply(
    lambda x: pd.Series(extract_engine_features(x))
)
drop_columns = ['engine', 'model']
dt.drop(drop_columns, axis=1, inplace=True)

dt.fillna(dt.mean(), inplace=True)

dt.sample(15)

,id,brand,model_year,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,car_age,Luxurious,HP,Liters,Cylinders,Turbocharged
36201,224734,27,2007,122141,2,31,302,10,1,0,16.00000,0,380.000000,4.6,8.000000,0.0
108046,296579,14,2019,24882,2,20,29,31,0,0,9.00000,0,480.000000,5.0,8.000000,0.0
101449,289982,3,2018,49500,3,31,29,31,1,0,13.00000,0,335.000000,3.0,6.000000,0.0
122331,310864,4,2022,11000,2,23,38,14,1,0,6.00000,0,560.000000,4.4,8.000000,0.0
101762,290295,36,2023,989,2,36,127,14,1,0,9.00000,0,342.797283,2.0,6.372479,1.0
81661,270194,17,2003,190640,2,12,261,14,0,0,2.00000,0,115.000000,1.7,4.000000,0.0
70853,259386,21,2023,20500,2,38,127,14,1,0,8.85398,1,550.000000,5.0,8.000000,0.0
56283,244816,39,2008,103990,2,38,38,14,1,0,8.00000,0,239.000000,4.0,6.000000,0.0
110516,299049,42,2016,79785,2,32,225,14,1,1,14.00000,0,342.797283,3.0,6.372479,1.0
100920,289453,14,2007,133510,2,20,127,14,0,0,4.00000,0,412.000000,5.0,8.000000,0.0


In [26]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            125690 non-null  int64  
 1   brand         125690 non-null  int64  
 2   model_year    125690 non-null  int64  
 3   milage        125690 non-null  int64  
 4   fuel_type     125690 non-null  int64  
 5   transmission  125690 non-null  int64  
 6   ext_col       125690 non-null  int64  
 7   int_col       125690 non-null  int64  
 8   accident      125690 non-null  int64  
 9   clean_title   125690 non-null  int64  
 10  car_age       125690 non-null  float64
 11  Luxurious     125690 non-null  int64  
 12  HP            125690 non-null  float64
 13  Liters        125690 non-null  float64
 14  Cylinders     125690 non-null  float64
 15  Turbocharged  125690 non-null  float64
dtypes: float64(5), int64(11)
memory usage: 15.3 MB


## Sample Submission File

In [27]:
sample_submission_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/sample_submission.csv?raw=true'

sf=pd.read_csv(sample_submission_url)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 162610 entries, 0 to 188532
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            162610 non-null  int64  
 1   brand         162610 non-null  int64  
 2   model_year    162610 non-null  int64  
 3   milage        162610 non-null  int64  
 4   fuel_type     162610 non-null  int64  
 5   transmission  162610 non-null  int64  
 6   ext_col       162610 non-null  int64  
 7   int_col       162610 non-null  int64  
 8   accident      162610 non-null  int64  
 9   clean_title   162610 non-null  int64  
 10  price         162610 non-null  int64  
 11  car_age       162610 non-null  int64  
 12  Luxurious     162610 non-null  int64  
 13  HP            162610 non-null  float64
 14  Liters        162610 non-null  float64
 15  Cylinders     162610 non-null  float64
 16  Turbocharged  162610 non-null  float64
dtypes: float64(4), int64(13)
memory usage: 26.4 MB


In [29]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            125690 non-null  int64  
 1   brand         125690 non-null  int64  
 2   model_year    125690 non-null  int64  
 3   milage        125690 non-null  int64  
 4   fuel_type     125690 non-null  int64  
 5   transmission  125690 non-null  int64  
 6   ext_col       125690 non-null  int64  
 7   int_col       125690 non-null  int64  
 8   accident      125690 non-null  int64  
 9   clean_title   125690 non-null  int64  
 10  car_age       125690 non-null  float64
 11  Luxurious     125690 non-null  int64  
 12  HP            125690 non-null  float64
 13  Liters        125690 non-null  float64
 14  Cylinders     125690 non-null  float64
 15  Turbocharged  125690 non-null  float64
dtypes: float64(5), int64(11)
memory usage: 15.3 MB


In [30]:
id = sf.pop('id')
y_pred = model.predict(dt)

# Create a submission DataFrame
submission_df = pd.DataFrame({
    'id': id,
    'class': y_pred
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('results.csv', index=False)
print("Submission file created: submission_file.csv")

Submission file created: submission_file.csv
